# Generate query image path list from Oxford5k queries

This script generate `image path list` file which is used in colmap's image retrieval system. 
You have to feed it to the image retrieval system to get `rank list` files for each query. 

In [ ]:
import os
from oxford_eval import get_query_image_filenames
from pprint import pprint
%matplotlib inline

GT_DIR = "./oxford5k_gt"
IMAGE_DIR = "./oxford5k_images"
OUTPUT_QUERY_IMAGE_DIR = "./oxford5k_query_images"
OUTPUT_QUERY_IMAGE_LIST_PATH = "./oxford5k_query_image_list.txt"
crop = False

# OUTPUT_QUERY_IMAGE_DIR = "./oxford5k_query_images_crop"
# OUTPUT_QUERY_IMAGE_LIST_PATH = "./oxford5k_query_image_crop_list.txt"
# crop = True

if not os.path.exists(OUTPUT_QUERY_IMAGE_DIR):
    os.mkdir(OUTPUT_QUERY_IMAGE_DIR)

query_names = get_query_image_filenames(GT_DIR)    
print("{} queres exists".format(len(query_names)))
# pprint(query_names)

prepare_query_images(query_names, GT_DIR, OUTPUT_QUERY_IMAGE_DIR, OUTPUT_QUERY_IMAGE_LIST_PATH, crop=crop, show_image=False)
print("done")

# Compute mAP with rank list files

In [ ]:
import os
import subprocess
from subprocess import CalledProcessError
import shlex

EXE_DIR_PATH = "."
WORK_DIR = "../ir_demo/"

def run_cmd(cmd, cwd=None):
    """
     limitation: cannot see progress until the process is finished. Not suitable for long-running task. 
    """
    try:       
        output = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT, cwd=cwd)
        output = output.decode('utf-8')
    except CalledProcessError as e:
        print(e)
        print("output: {}".format(e.output.decode('utf-8')))
    return output
        

# cmd = "ls {EXE_DIR_PATH}".format( \
#         EXE_DIR_PATH=EXE_DIR_PATH)
# output = run_cmd(cmd)
# print(output)

In [ ]:
## COLMAP CBIR
# Report these only. Query will be always cropped.
# QUERY_RESULT_PATH = "../tutorial/query_result_crop_vocab_64k_norerank.txt" # 57.43
# QUERY_RESULT_PATH = "../tutorial/query_result_crop_vocab_64k_geomverif.txt" # 67.90
# QUERY_RESULT_PATH = "../tutorial/query_result_crop_vocab_200k_norerank.txt" # 60.93
# QUERY_RESULT_PATH = "../tutorial/query_result_crop_vocab_200k_geomverif.txt" # 69.67
# QUERY_RESULT_PATH = "../tutorial/query_result_crop_vocab_256k_norerank.txt" # 61.02
# QUERY_RESULT_PATH = "../tutorial/query_result_crop_vocab_256k_geomverif.txt" # 67.94

## Geometric Burstiness CBIR
# QUERY_RESULT_PATH = "../../geometric_burstiness/tutorial/oxf5k_demo.retrieval_ranking.txt.colmap" # 70.02
# QUERY_RESULT_PATH = "../../geometric_burstiness/tutorial/oxf5k_demo.inlier_count_ranking.txt.colmap" # 67.89
# QUERY_RESULT_PATH = "../../geometric_burstiness/tutorial/oxf5k_demo.effective_inlier_count_ranking.txt.colmap" # 68.18
# QUERY_RESULT_PATH = "../../geometric_burstiness/tutorial/oxf5k_demo.inter_image_geometric_burstiness_ranking.txt.colmap" # 67.37

import shutil
OUTPUT_RANK_LIST_DIR = "./oxford5k_rank_list"
if not os.path.exists(OUTPUT_RANK_LIST_DIR):
    os.mkdir(OUTPUT_RANK_LIST_DIR)

fh = None
result_count = 0
query_count = 0

if os.path.exists(OUTPUT_RANK_LIST_DIR):
    shutil.rmtree(OUTPUT_RANK_LIST_DIR)
os.mkdir(OUTPUT_RANK_LIST_DIR)

with open(QUERY_RESULT_PATH, "r") as f:
    for line_idx, line in enumerate(f.readlines()):
        # print(line)
        if "Querying for image " in line:
            query_name = line.split(" ")[3]            
            if not fh is None:
                # print("write rank with {} images to filepath: {}".format(result_count, rank_filepath))
                query_count += 1
                fh.close()
            rank_filepath = os.path.join(OUTPUT_RANK_LIST_DIR, query_name+"_rank.txt")
            fh = open(rank_filepath, "w")
            result_count = 0
        else:      
            result_image_filename = line.strip().split(", ")[1].split("=")[1]
            result_image_name = result_image_filename.replace(".jpg", "")
            result_count += 1
            fh.write(result_image_name+"\n")
    if not fh is None:
        # print("write rank with {} images to filepath: {}".format(result_count, rank_filepath))
        query_count += 1
        fh.close()
        
print("{} query results are ready for evaluation.".format(query_count))
            
OXFORD_GT_WORKDIR = "./oxford5k_gt/"
# Relative PATH according to above work dir
RANK_LIST_DIR = "./oxford5k_rank_list"
COMPUTE_AP_EXE_PATH = "./oxford5k_compute_ap"

files = os.listdir(RANK_LIST_DIR)
ap_sum = 0.0
for f in files:
    query_name = f.split(".")[0]
    
    cmd = "{COMPUTE_AP_EXE_PATH} {QUERY_NAME} {RANK_LIST_FILE_PATH}".format(\
        COMPUTE_AP_EXE_PATH=COMPUTE_AP_EXE_PATH, \
        QUERY_NAME = query_name, \
        RANK_LIST_FILE_PATH = os.path.abspath(os.path.join(RANK_LIST_DIR, f)))
    output = run_cmd(cmd, cwd=OXFORD_GT_WORKDIR)
    # print("cmd: ", cmd)
    # print("AP: ", output)
    ap = float(output)
    ap_sum += ap
    
ap_mean = ap_sum / len(files)
print("mAP:", ap_mean)